# QiCore Base - Framework for Modern TypeScript Development

**The reality of modern development:**
- APIs everywhere: REST, GraphQL, MCP servers, LLM APIs
- Data sources: databases, files, streams, external services  
- AI era integration: agents, toolsets, service orchestration
- Internet-first applications with multiple failure points

**The need:** A lightweight, extensible framework to handle failures systematically.

**What `@qi/base` provides:**
- **Result<T>**: Generic pattern that works across all integration scenarios
- **QiError framework**: Base structure that you extend for your specific domains
- **Lightweight**: No heavy dependencies, works everywhere
- **Extensible**: Build domain-specific error handling on the foundation

**Working directory:** `typescript/docs/tutorial/nb/`

## Setup and Environment

Ensure you're running this notebook from the correct working directory:

```bash
cd typescript/docs/tutorial/nb
jupyter lab  # or code . for VSCode
```

**Required setup:**
1. QiCore packages built: `cd typescript && bun run build`
2. Deno Jupyter kernel installed: `deno jupyter --install`
3. Working directory: `typescript/docs/tutorial/nb/`

Verify your environment:

In [ ]:
// Verify TypeScript and imports work
console.log("Working directory:", Deno.cwd())
console.log("TypeScript version:", (globalThis as any).Deno?.version?.typescript || "unknown")

// Test @qi/base import
try {
    const { success } = await import("@qi/base")
    console.log("✅ @qi/base import successful")
    console.log("Sample Result:", success(42))
} catch (error) {
    console.error("❌ Import failed:", error.message)
    console.log("Check: bun run build in typescript/ directory")
}

## The Challenge: Multiple Services, Multiple Failures

Modern applications integrate with numerous external dependencies:

```typescript
// Your typical application calls
await fetchUserProfile(userId)    // Auth service - can fail
await getUserPreferences(userId)  // Database - can fail  
await logEvent(event)             // Analytics API - can fail
await processPayment(amount)      // Payment gateway - can fail
await sendNotification(message)   // Email service - can fail
```

Each service can fail in different ways. How do you handle this systematically without ending up with inconsistent error handling across your codebase?

In [ ]:
// Example: Exception-based error handling (problematic)
interface User {
    id: string
    name: string
    email: string
}

function getUserById(id: string): User {
    if (!id) {
        throw new Error("ID required")  // Hidden in signature
    }
    if (id === "404") {
        throw new Error("User not found")  // Hidden in signature
    }
    if (id === "500") {
        throw new Error("Database error")  // Hidden in signature
    }
    
    return { id, name: `User ${id}`, email: `user${id}@example.com` }
}

// Usage requires defensive programming
try {
    const user = getUserById("123")
    console.log("Success:", user)
} catch (error) {
    console.log("Error:", error.message)
    // How do we know what kind of error this is?
    // Should we retry? Fix input? Report to user?
}

## QiCore Base: Framework Architecture

**Two-layer design:**

### 1. Result<T> - The Generic Container
```typescript
type Result<T, E = QiError> = 
  | { readonly tag: 'success'; readonly value: T }
  | { readonly tag: 'failure'; readonly error: E }
```
Works with any data type, any error type. Handles all your integration scenarios.

### 2. QiError - The Framework Foundation
```typescript
interface QiError {
  readonly code: string
  readonly message: string  
  readonly category: ErrorCategory
  readonly context: Record<string, unknown>
}
```

**Critical insight:** QiError is a *framework* that you extend. `@qi/base` sets up the structure - you build your domain-specific errors on top.

In [ ]:
// Import core Result types and functions
import { 
    Result, success, failure, match, 
    validationError, systemError, businessError,
    type QiError 
} from '@qi/base'

// Result-based implementation - explicit error handling
function getUserByIdSafe(id: string): Result<User, QiError> {
    if (!id) {
        return failure(validationError("ID required"))
    }
    if (id === "404") {
        return failure(businessError("User not found"))
    }
    if (id === "500") {
        return failure(systemError("Database error"))
    }
    
    return success({ id, name: `User ${id}`, email: `user${id}@example.com` })
}

// Function signature now explicitly shows it can fail:
// getUserByIdSafe(id: string): Result<User, QiError>

console.log("Function signature explicitly shows failure cases")
console.log("Type system enforces error handling")

In [ ]:
// Pattern matching for exhaustive case handling
const result1 = getUserByIdSafe("123")
const result2 = getUserByIdSafe("")
const result3 = getUserByIdSafe("404")

// match() ensures both success and failure cases are handled
match(
    (user) => console.log("Success:", user),
    (error) => console.log("Error:", error.message, "Category:", error.category),
    result1
)

match(
    (user) => console.log("Success:", user),
    (error) => console.log("Error:", error.message, "Category:", error.category),
    result2
)

match(
    (user) => console.log("Success:", user),
    (error) => console.log("Error:", error.message, "Category:", error.category),
    result3
)

## The Framework in Action: Domain-Specific Extensions

QiError provides the base framework. Each domain extends it with specific categories and context:

### Payment Domain Extension
```typescript
interface PaymentError extends QiError {
  category: 'VALIDATION' | 'NETWORK' | 'BUSINESS'  // Domain-specific subset
  context: {
    paymentId?: string
    amount?: number
    provider?: 'stripe' | 'paypal'
    declineReason?: string
  }
}
```

### API Integration Extension  
```typescript
interface APIError extends QiError {
  category: 'NETWORK' | 'AUTHENTICATION' | 'SYSTEM'
  context: {
    endpoint: string
    method: string
    statusCode?: number
    retryAttempt?: number
  }
}
```

The framework approach: consistent structure across all domains.

In [ ]:
// Building domain-specific errors on the QiError framework
import { QiError, ErrorCategory } from '@qi/base'

// Payment domain extension
interface PaymentError extends QiError {
    category: 'VALIDATION' | 'NETWORK' | 'BUSINESS'
    context: {
        paymentId?: string
        amount?: number
        provider?: string
        declineReason?: string
    }
}

// Factory function following the framework pattern
function createPaymentError(
    code: string,
    message: string,
    category: PaymentError['category'],
    context: PaymentError['context'] = {}
): PaymentError {
    return { code, message, category, context }
}

// Usage examples
const errors = [
    createPaymentError('INSUFFICIENT_FUNDS', 'Card declined', 'BUSINESS', {
        paymentId: 'pay_123',
        amount: 5000,
        declineReason: 'insufficient_funds'
    }),
    createPaymentError('GATEWAY_TIMEOUT', 'Payment gateway timeout', 'NETWORK', {
        paymentId: 'pay_123',
        provider: 'stripe'
    }),
    createPaymentError('INVALID_AMOUNT', 'Amount must be positive', 'VALIDATION', {
        amount: -100
    })
]

console.log("Domain-specific errors built on QiError framework:")
errors.forEach(error => {
    console.log(`${error.code}: ${error.message} [${error.category}]`)
    console.log(`Context:`, error.context)
})

## Functional Composition Patterns

Result<T> supports functional composition through map, flatMap, and other combinators. This enables safe chaining of operations that might fail.

In [ ]:
// Functional composition with map and flatMap
import { map, flatMap } from '@qi/base'

// map: transform success values, pass through errors
const result = success(42)
const doubled = map((x: number) => x * 2, result)
const stringified = map((x: number) => `Value: ${x}`, doubled)

console.log("map chain:", stringified)

// flatMap: chain operations that return Results
function parseNumber(s: string): Result<number, QiError> {
    const n = parseInt(s, 10)
    if (isNaN(n)) {
        return failure(validationError("Not a number"))
    }
    return success(n)
}

function sqrt(n: number): Result<number, QiError> {
    if (n < 0) {
        return failure(validationError("Cannot sqrt negative"))
    }
    return success(Math.sqrt(n))
}

// Chain operations - error in any step stops the chain
const chainResult = flatMap(sqrt, parseNumber("16"))
console.log("Chain success:", chainResult)

const chainError = flatMap(sqrt, parseNumber("invalid"))
console.log("Chain error:", chainError)

## Real-World Integration: API Client with Framework

Demonstrating how Result<T> + QiError framework handles actual service integration scenarios.

In [ ]:
// Real-world API integration using the framework
import { Result, success, failure, match, flatMap } from '@qi/base'

// API domain extension of QiError framework
interface APIError extends QiError {
    category: 'NETWORK' | 'AUTHENTICATION' | 'VALIDATION' | 'SYSTEM'
    context: {
        endpoint: string
        method: string
        statusCode?: number
        responseTime?: number
    }
}

function createAPIError(
    code: string,
    message: string,
    category: APIError['category'],
    context: Partial<APIError['context']>
): APIError {
    return { 
        code, 
        message, 
        category, 
        context: { endpoint: '', method: '', ...context }
    }
}

// Simulate API calls with realistic failure scenarios
async function callExternalAPI(endpoint: string): Promise<Result<any, APIError>> {
    const startTime = Date.now()
    
    // Simulate different failure modes
    if (endpoint.includes('auth')) {
        return failure(createAPIError(
            'AUTH_FAILED',
            'Invalid API key',
            'AUTHENTICATION',
            { endpoint, method: 'GET', statusCode: 401 }
        ))
    }
    
    if (endpoint.includes('timeout')) {
        return failure(createAPIError(
            'REQUEST_TIMEOUT',
            'Request exceeded 30s timeout',
            'NETWORK',
            { endpoint, method: 'POST', responseTime: 30000 }
        ))
    }
    
    if (endpoint.includes('server-error')) {
        return failure(createAPIError(
            'INTERNAL_ERROR',
            'Database connection failed',
            'SYSTEM',
            { endpoint, method: 'GET', statusCode: 500 }
        ))
    }
    
    // Success case
    return success({ 
        data: `Response from ${endpoint}`,
        timestamp: new Date().toISOString()
    })
}

// Compose multiple API calls using Result<T> framework
async function fetchUserDashboard(userId: string): Promise<Result<any, APIError>> {
    const profileResult = await callExternalAPI(`/users/${userId}/profile`)
    if (profileResult.tag === 'failure') return profileResult
    
    const prefsResult = await callExternalAPI(`/users/${userId}/preferences`)  
    if (prefsResult.tag === 'failure') return prefsResult
    
    const analyticsResult = await callExternalAPI(`/analytics/${userId}`)
    if (analyticsResult.tag === 'failure') return analyticsResult
    
    return success({
        profile: profileResult.value,
        preferences: prefsResult.value,
        analytics: analyticsResult.value
    })
}

// Test with different scenarios
const testScenarios = [
    'user123',           // Success
    'auth-fail',         // Auth error
    'timeout-user',      // Network timeout  
    'server-error-user'  // System error
]

for (const userId of testScenarios) {
    console.log(`\nFetching dashboard for: ${userId}`)
    const result = await fetchUserDashboard(userId)
    
    match(
        (dashboard) => console.log('✅ Dashboard loaded successfully'),
        (error) => {
            console.log(`❌ Failed: ${error.message}`)
            console.log(`   Category: ${error.category}`)  
            console.log(`   Context:`, error.context)
            
            // Framework enables consistent error handling strategies
            switch (error.category) {
                case 'NETWORK':
                    console.log('   → Strategy: Retry with exponential backoff')
                    break
                case 'AUTHENTICATION':
                    console.log('   → Strategy: Redirect to login')
                    break
                case 'SYSTEM':
                    console.log('   → Strategy: Show maintenance message')
                    break
            }
        },
        result
    )
}

## Framework Benefits: Why This Approach Works

**Lightweight & Extensible:**
- Core framework in `@qi/base` stays minimal
- Each domain builds specific extensions 
- No heavyweight dependencies or complex abstractions

**Systematic Error Handling:**
- Consistent structure across all domains
- Categories drive retry strategies automatically  
- Rich context enables effective debugging

**Integration-Ready:**
- Works with any external service or data source
- Handles the reality of multiple failure points
- Scales from simple APIs to complex service orchestration

**Modern Development Alignment:**
- Perfect for AI era: MCP servers, LLM APIs, agent toolsets
- TypeScript ecosystem integration
- Framework approach that grows with your application

## Next Steps

- **API Reference:** [docs/api/base/](../../api/base/) - Complete Result<T> and QiError documentation
- **Usage Patterns:** [docs/tutorial/qi-base.md](../qi-base.md) - Real-world patterns and best practices  
- **Working Examples:** [typescript/app/](../../../app/) - Production-ready implementations

**Continue the series:** [02-qi-core.ipynb](./02-qi-core.ipynb) - Configuration, logging, and caching built on this foundation

In [ ]:
// Pattern 3: Domain-Specific Extensions
console.log('=== Pattern 3: Domain-Specific Extensions ===')

// Each domain extends QiError with its own context and categories
console.log('Domain Extension Template:')
console.log(`
interface DomainError extends QiError {
  category: 'SUBSET_OF_ERROR_CATEGORIES'  // Choose relevant categories
  context: {
    // Domain-specific fields
    entityId?: string
    operation?: string
    businessData?: any
  }
}

function createDomainError(
  code: string,
  message: string, 
  category: DomainError['category'],
  context: DomainError['context'] = {}
): DomainError { /* ... */ }
`)

// Show multiple domain examples
const domainExamples = {
  'Payment': {
    categories: ['VALIDATION', 'NETWORK', 'BUSINESS'],
    context: ['paymentId', 'amount', 'provider', 'cardLast4']
  },
  'User Management': {
    categories: ['VALIDATION', 'AUTHENTICATION', 'AUTHORIZATION', 'BUSINESS'],
    context: ['userId', 'email', 'operation', 'requiredRole']
  },
  'Order Processing': {
    categories: ['VALIDATION', 'BUSINESS', 'SYSTEM'],
    context: ['orderId', 'customerId', 'items', 'stage']
  },
  'File Operations': {
    categories: ['VALIDATION', 'SYSTEM', 'RESOURCE'],
    context: ['filePath', 'fileSize', 'operation', 'permissions']
  }
}

console.log('\nDomain Extension Examples:')
Object.entries(domainExamples).forEach(([domain, spec]) => {
  console.log(`\n${domain}:`)
  console.log(`  Categories: ${spec.categories.join(', ')}`)
  console.log(`  Context: ${spec.context.join(', ')}`)
})

console.log('\nPattern Benefits:')
console.log('- Domain-specific debugging context')
console.log('- Type safety per business domain')
console.log('- Scalable error handling architecture')
console.log('- Clear separation of concerns')

## Summary: The QiCore Base System

After exploring contracts, logic, use cases, and patterns, we can see that `@qi/base` provides:

### **Core Architecture**
- **Result<T, E>**: Makes errors explicit in function signatures
- **QiError**: Provides structured, categorized error information  
- **Functional Operations**: Enable safe composition and transformation

### **Key Patterns**
1. **Explicit Error Contracts**: Every fallible function returns `Result<T, E>`
2. **Category-Based Strategies**: Error categories determine retry/response logic
3. **Domain Extensions**: Each business domain extends `QiError` with specific context
4. **Functional Composition**: Chain operations with automatic error propagation

### **Benefits**
- **Type Safety**: Compile-time error checking
- **Systematic Handling**: Consistent error strategies
- **Rich Context**: Structured debugging information
- **Composability**: Clean operation chaining
- **Scalability**: Domain-specific error handling

This foundation enables building reliable TypeScript applications with predictable, maintainable error handling.